Alhassan, Abdulaziz and Sevtsuk, Andres, Madina Python Package: Scalable Urban Network Analysis for Modeling Pedestrian and Bicycle Trips in Cities. Available at SSRN: https://ssrn.com/abstract=4748255 or http://dx.doi.org/10.2139/ssrn.4748255

In [212]:
import madina as md
import geopandas as gpd
import pandas as pd
import math
from shapely.geometry import box
import madina.una.tools as una
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from concurrent.futures import ProcessPoolExecutor

In [ ]:
#collison['DATETIME'] = pd.to_datetime(collison['CRASH DATE'] + ' ' + collison['CRASH TIME'])
#collison21_23 = collison.loc[(collison['DATETIME'] > pd.Timestamp('2021-1-1 00:00'))]
#collison21_23.iloc[:,:-1].to_file('../01_data/03_intermediate/Collision_21_23/Collision.shp', driver='ESRI Shapefile')

In [199]:
nycstreet = gpd.read_file("../01_data/03_intermediate/cut/cut.shp")

In [200]:
collison = gpd.read_file('../01_data/03_intermediate/Collision_21_23/Collision.shp')
collison['DATETIME'] = pd.to_datetime(collison['CRASH DATE'] + ' ' + collison['CRASH TIME'])

In [205]:
collison22_23 = collison.loc[(collison['DATETIME'] > pd.Timestamp('2022-1-1 00:00'))]
collison22_23.shape
collison = collison22_23

----

In [206]:
delta_lat = 2 / 111  
delta_lon = 2 / (111 * math.cos(math.radians(latitude)))

In [216]:
nycstreet.sindex
collison.sindex

get_previous_index = pd.DataFrame() 

In [218]:
for i in range(0,collison.shape[0]):

    try:

        origin = collison.iloc[i:i+1]

        # time
        mask = (collison['DATETIME'] >= origin['DATETIME'].iloc[0] - pd.Timedelta(days=30)) & (collison['DATETIME'] < origin['DATETIME'].iloc[0])
        
        # space
        min_lat = origin['LATITUDE'] - delta_lat
        max_lat = origin['LATITUDE'] + delta_lat
        min_lon = origin['LONGITUDE'] - delta_lon
        max_lon = origin['LONGITUDE'] + delta_lon
        bounding_box = box(min_lon, min_lat, max_lon, max_lat)

        #slice
        destination = collison.loc[mask]
        space = nycstreet[nycstreet['geometry'].intersects(bounding_box)]
        destination = destination[destination['geometry'].within(bounding_box)]

        if destination.shape[0] == 0 or space.shape[0] == 0:

            gdf = origin.copy()
            gdf = gdf.to_crs('EPSG:32619')
            gdf['gravity_to_previous'] = 0
        
        else:

            # weight
            destination['duration'] = (origin['DATETIME'].iloc[0] - destination['DATETIME']).dt.total_seconds() / 3600 # hour
            destination['weight'] = 1/destination['duration'] 

            # prejection
            space = space.to_crs('EPSG:32619')
            origin = origin.to_crs('EPSG:32619')
            destination = destination.to_crs('EPSG:32619')

            # Load Layer
            nyc = md.Zonal()
            nyc.load_layer(name='street', source = space)
            nyc.load_layer('collison', source = origin)
            nyc.load_layer('time_lag', source = destination)

            # Creating a network, and adding origins and destinations
            nyc.create_street_network(source_layer="street")
            nyc.insert_node(label='origin', layer_name="collison")
            nyc.insert_node(label='destination', layer_name="time_lag")

            nyc.create_graph()

            una.accessibility(
                nyc,
                search_radius=3000,
                beta=0.001,
                save_gravity_as='gravity_to_previous',
                destination_weight= 'duration'
            )

            gdf = nyc['collison'].gdf

        get_previous_index = pd.concat([get_previous_index, gdf], ignore_index = True)

    except Exception as e:
        print(f"Error processing record {i}: {e}")
        error_records = pd.concat([error_records, origin], ignore_index=True)

Error processing record 103: 1961ns (100.00%)]
Error processing record 290: 'gravity'00.00%)]
Error processing record 419: 610ins (100.00%)]
Error processing record 468: 858ins (100.00%)]
Error processing record 704: 1176ns (100.00%)]
Error processing record 800: 'gravity'00.00%)]
Error processing record 938: 11gins (100.00%)]
Error processing record 972: 603ins (100.00%)]
Error processing record 1063: 152ns (100.00%)]
Error processing record 1114: 'gravity'0.00%)]
Error processing record 1191: 537ns (100.00%)]
Error processing record 1223: 'gravity'0.00%)]
Error processing record 1294: 1256s (100.00%)]
Error processing record 1437: 867ns (100.00%)]
Error processing record 1566: 23ins (100.00%)]
Error processing record 1607: 1288s (100.00%)]
Error processing record 1660: 789ns (100.00%)]
Error processing record 1726: 218ns (100.00%)]
Error processing record 1745: 843ns (100.00%)]
Error processing record 1780: 749ns (100.00%)]
Error processing record 1793: 387ns (100.00%)]
Error process

In [222]:
index = get_previous_index.drop('DATETIME',axis=1)

In [225]:
index.to_file('../01_data/03_intermediate/index/index.shp', driver='ESRI Shapefile')

/var/folders/xn/k3tchqyx5bn081b3tlx0k4940000gn/T/ipykernel_29896/2206586428.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  index.to_file('../01_data/03_intermediate/index/index.shp', driver='ESRI Shapefile')
